In [ ]:
a = [1,1,2,3,4,5,6,6,6,6,2]
a.pop(1)
list(filter(lambda a: a != 1, a))

[2, 3, 4, 5, 6, 6, 6, 6, 2]

In [ ]:
a = {'player1':None, 'player2':None, 'player3':None, 'player4':None}

In [ ]:
class groupCards:
  def __init__(self):
    self.card_numbers = {
        '2':0,'3':1,'4':2,'5':3,'6':4,'7':5,'8':6,'9':7,'10':8,
        'J':9,'Q':10,'K':11,'A':12
    }
    self.card_numbers_invert = {v:k for k, v in self.card_numbers.items()}

    self.straight_sets = self._create_straight_sets()

  def _create_straight_sets(self):
    nums = list(self.card_numbers.values())
    st = []
    for i in range(len(nums)-5,-1,-1):  # Sorted from maximum straight
      s = []
      for j in range(i,i+5):
        s.append(nums[j])
      st.append(set(s))
    st.append({12,0,1,2,3})  # Ace,2,3,4,5
    return st

  def set_number_to_cards_value(self, c):
    return self.card_numbers[c[1:]]

  def max(self):
    self.all_floor_cards = sum(self.players_cards.values(), [])
    cards = [card[1:] for card in self.all_floor_cards]
    maximum = max([self.card_numbers[card] for card in cards])
    self.players_max = [card for card in self.all_floor_cards if self.card_numbers[card[1:]] == maximum]

  def high_cards(self, players_cards):
    self.max()
    winners = []
    for player, cards in players_cards.items():
      for card in cards:
        if card in self.players_max:
          winners.append(player)
    return winners

  def check_high_card(self):
    winners = self.high_cards(self.players_cards)
    for _ in range(4):
      new_cards = {player:self.players_cards[player] for player in winners}
      winners = self.high_cards(new_cards)
    return winners[0]

  def one_pair_for_a_player(self, player_card, numbers=None):
    if numbers is None:
      player_card_with_floor_cards = player_card+self.floor_cards
      numbers = [c[1:] for c in player_card_with_floor_cards]
    for i, n in enumerate(numbers[:-1]):
      if n in numbers[i+1:]:
        return True, n
    return False, None

  def check_one_pair(self):
    pair_cards_ = {'player1':None, 'player2':None, 'player3':None, 'player4':None}
    for p, cards in self.players_cards.items():
      ispair, pair_card = self.one_pair_for_a_player(cards)
      if ispair:
        pair_cards_[p] = pair_card
    pair_cards = {}
    for p, c in pair_cards_.items():
      if c is not None:
        pair_cards[p] = c
    del pair_cards_
    if pair_cards:
      return max(pair_cards, key=lambda x: self.card_numbers[pair_cards[x]])
    else:
      return None

  def two_pairs_for_a_player(self, player_card):
    player_card_with_floor_cards = player_card+self.floor_cards
    numbers = [c[1:] for c in player_card_with_floor_cards]
    count = {}
    for num in numbers:
      if num in count:
        count[num] += 1
      else:
        count[num] = 1
    pairs = [num for num, cnt in count.items() if cnt >= 2]
    if len(pairs)==2:
      return True, pairs
    else:
      return False, None

  def check_two_pairs(self):
    pair_cards_ = {'player1':None, 'player2':None, 'player3':None, 'player4':None}
    for p, cards in self.players_cards.items():
      ispair, pair_card = self.two_pairs_for_a_player(cards)
      if ispair:
        pair_cards_[p] = pair_card
    pair_cards = {}
    for p, c in pair_cards_.items():
      if c is not None:
        pair_cards[p] = c
    del pair_cards_
    if pair_cards:
      maximum = max(sum(pair_cards.values(), []))
      for p, c in pair_cards.items():
        if maximum in c:
          return p
    else:
      return None

  def three_of_a_kind_for_a_player(self, player_card):
    player_card_with_floor_cards = player_card+self.floor_cards
    numbers = [c[1:] for c in player_card_with_floor_cards]
    count = {}
    for num in numbers:
      if num in count:
        count[num] += 1
      else:
        count[num] = 1
    pairs = [num for num, cnt in count.items() if cnt >= 3]
    if len(pairs):
      return True, pairs[0]
    else:
      return False, None

  def check_three_of_a_kind(self):
    pair_cards_ = {'player1':None, 'player2':None, 'player3':None, 'player4':None}
    for p, cards in self.players_cards.items():
      ispair, pair_card = self.three_of_a_kind_for_a_player(cards)
      if ispair:
        pair_cards_[p] = pair_card
    pair_cards = {}
    for p, c in pair_cards_.items():
      if c is not None:
        pair_cards[p] = c
    del pair_cards_
    if pair_cards:
      return max(pair_cards, key=lambda x: self.card_numbers[pair_cards[x]])
    else:
      return None

  def straight_for_a_player(self, player_card, player_card_with_floor_cards=None):
    if player_card_with_floor_cards is None:
      player_card_with_floor_cards = player_card+self.floor_cards
    numbers = sorted([self.card_numbers[c[1:]] for c in player_card_with_floor_cards])

    for st in self.straight_sets:
      if st.issubset(numbers):
        if st=={12,0,1,2,3}:  # Ace,2,3,4,5
          return True, 3
        return True, max(st)
    return False, None

  def check_straight(self):
    straight_cards_ = {'player1':None, 'player2':None, 'player3':None, 'player4':None}
    for p, cards in self.players_cards.items():
      ispair, straight_card = self.straight_for_a_player(cards)
      if ispair:
        straight_cards_[p] = straight_card
    straight_cards = {}
    for p, c in straight_cards_.items():
      if c is not None:
        straight_cards[p] = c
    del straight_cards_
    if straight_cards:
      return max(straight_cards, key=lambda x: straight_cards[x])
    else:
      return None

  def flush_for_a_player(self, player_card):
    player_card_with_floor_cards = player_card+self.floor_cards
    groups = [c[0] for c in player_card_with_floor_cards]
    for g in ["C", "D", "S", "H"]:
      if groups.count(g)>=5:
        maximum = max([self.card_numbers[c[1:]] for c in player_card_with_floor_cards if c.startswith(g)])
        return True, maximum

    return False, None

  def check_flush(self):
    flush_cards_ = {'player1':None, 'player2':None, 'player3':None, 'player4':None}
    for p, cards in self.players_cards.items():
      ispair, flush_card = self.flush_for_a_player(cards)
      if ispair:
        flush_cards_[p] = flush_card
    flush_cards = {}
    for p, c in flush_cards_.items():
      if c is not None:
        flush_cards[p] = c
    del flush_cards_
    if flush_cards:
      return max(flush_cards, key=lambda x: flush_cards[x])
    else:
      return None

  def full_house_for_a_player(self, player_card):
    player_card_with_floor_cards = player_card+self.floor_cards
    numbers = sorted([c[1:] for c in player_card_with_floor_cards])
    isthree, three_card = self.three_of_a_kind_for_a_player(player_card)
    ispair = False
    if isthree:
      numbers = list(filter(lambda a: a != three_card, numbers)) # remove three cards
      ispair, pair_card = [[True,  n] for n in set(numbers) if numbers.count(n)>=2][0]
      if ispair:
        return True, self.card_numbers[three_card], self.card_numbers[pair_card]

    return False, None, None

  def check_full_house(self):
    full_house_cards = {'player1':None, 'player2':None, 'player3':None, 'player4':None}
    full_exists = False
    max_three = -1
    max_pair = -1
    for p, cards in self.players_cards.items():
      isfullhouse, three_card, pair_card = self.full_house_for_a_player(cards)
      if isfullhouse:
        full_exists = True
        full_house_cards[p] = [three_card, pair_card]
        max_three = max(three_card, max_three)
        max_pair = max(max_pair, pair_card)
    if full_exists:
      winner = []
      for p, cards in full_house_cards.items():
        if cards is not None and max_three==cards[0]:
          winner.append(p)
      if len(winner)==1:
        return winner[0]

      for p in winner:
        if max_pair==full_house_cards[p][1]:
          return p
    else:
      return None


  def four_of_a_kind_for_a_player(self, player_card):
    player_card_with_floor_cards = player_card+self.floor_cards
    numbers = sorted([self.card_numbers[c[1:]] for c in player_card_with_floor_cards])
    for n in set(numbers):
      if numbers.count(n)==4:
        return True, n
    return False, None

  def check_four_of_a_kind(self):
    four_kind_cards = {'player1':None, 'player2':None, 'player3':None, 'player4':None}
    four_exists = False
    max_four = -1
    for p, cards in self.players_cards.items():
      isfour, four_card = self.four_of_a_kind_for_a_player(cards)
      if isfour:
        four_exists = True
        max_four = max(max_four, four_card)
        four_kind_cards[p] = four_card
    if four_exists:
      for p, cards in four_kind_cards.items():
        if cards is not None and max_four==cards:
          return p
    else:
      return None

  def straight_flush_for_a_player(self, player_card):
    player_card_with_floor_cards = player_card+self.floor_cards
    groups = [c[0] for c in player_card_with_floor_cards]
    isflush = False
    flush_cards = []
    for g in ["C", "D", "S", "H"]:
      if groups.count(g)>=5:
        isflush = True
        break
    if isflush:
      for c in player_card_with_floor_cards:
        if c.startswith(g):
          flush_cards.append(c)
    else:
      return False, None
    return self.straight_for_a_player(None, flush_cards)

  def check_straight_flush(self):
    st_flush_cards = {'player1':None, 'player2':None, 'player3':None, 'player4':None}
    stfl_exists = False
    max_stfl = -1
    for p, cards in self.players_cards.items():
      is_stfl, stfl_card = self.straight_flush_for_a_player(cards)
      if is_stfl:
        stfl_exists = True
        max_stfl = max(max_stfl, stfl_card)
        st_flush_cards[p] = stfl_card
    if stfl_exists:
      for p, cards in st_flush_cards.items():
        if cards is not None and max_stfl==cards:
          return p
    else:
      return None

  def royal_flush_for_a_player(self, player_card):
    player_card_with_floor_cards = player_card+self.floor_cards
    groups = [c[0] for c in player_card_with_floor_cards]
    isflush = False
    flush_cards = []
    for g in ["C", "D", "S", "H"]:
      if groups.count(g)>=5:
        isflush = True
        break
    if isflush:
      for c in player_card_with_floor_cards:
        if c.startswith(g):
          flush_cards.append(c)
    else:
      return False
    numbers = sorted([self.card_numbers[c[1:]] for c in flush_cards])
    if {12,11,10,9,8}.issubset(numbers):
        return True
    return False

  def check_royal_flush(self):
    royal_exists = False
    max_royal = -1
    for p, cards in self.players_cards.items():
      is_royal = self.royal_flush_for_a_player(cards)
      if is_royal:
        return p
    return None

  def __call__(self, players_cards, floor_cards):
    self.players_cards = players_cards
    self.floor_cards = floor_cards



In [ ]:
group_cards = groupCards()
cards = {
    'player1' : ["D2", 'CQ'],
    'player2' : ["C5", 'S5'],
    'player3' : ["C9", 'S7'],
    'player4' : ["CJ", 'C10'],
}
group_cards(cards, ['CJ', 'CQ', 'C10'])

In [ ]:
# group_cards.royal_flush_for_a_player(cards['player4'])
group_cards.check_high_card()

'player1'